<a href="https://colab.research.google.com/github/saivenkatreddy29/Building-LLM-from-Scratch/blob/main/Chapter_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from importlib.metadata import version
print("torch version", version("torch"))

torch version 2.5.1+cu121


In [ ]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

Mathemetical dot product

In [ ]:
result = torch.zeros(len(inputs))
n = len(inputs)
query = inputs[1]
for i in range(n):
  result[i] = (sum(query*inputs[i]))
print(result)


tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


Using torch

In [ ]:
import numpy as np
query = inputs[1]
attention_scores = torch.zeros(len(inputs))
n = len(inputs)
for i in range(n):
  attention_scores[i] = torch.dot(query,inputs[i])
attention_scores

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [ ]:
attention_scores.shape,inputs.shape

(torch.Size([6]), torch.Size([6, 3]))

Using mathematical method for normalizing

In [ ]:
math_norm = attention_scores/sum(attention_scores)
math_norm

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])

Using softmax written in python

In [ ]:
import math
def naive_softmax(x):
  denom = 0
  for i in x:
    denom += math.exp(i)
  print(denom)
  # denom = sum(math.exp(x))
  return torch.exp(x)/denom
python_norm = naive_softmax(attention_scores)
print(python_norm)


18.745270996882674
tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


Using torch softmax

In [ ]:
normalized_scores = torch.softmax(attention_scores, dim = -1)
normalized_scores

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

Context vector for 2nd input we used as example

In [ ]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
print(context_vec_2)
for i in range(n):
  # print(f'i:{inputs[i]}')
  context_vec_2 += normalized_scores[i] * inputs[i]
print(context_vec_2)

tensor([0., 0., 0.])
tensor([0.4419, 0.6515, 0.5683])


In [ ]:
attn_score = inputs@inputs.T
print(attn_score)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [ ]:
attn_weights = torch.softmax(attn_score, dim = -1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [ ]:
print(attn_weights.sum(dim = -1))

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [ ]:
context_vector = attn_weights @ inputs
print(context_vector)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


# Implementing Self attention with trainable weights

In [ ]:
x2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [ ]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad = False)
W_key = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad = False)
W_value = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad = False)



In [ ]:
query2 = x2@ W_query
key_2 = x2@ W_key
value2 = x2@ W_value

print(query2)

tensor([0.4306, 1.4551])


In [ ]:
keys = inputs@W_key
values = inputs@W_value
print(f'key shape:{keys.shape}')
print(f'value shape:{values.shape}')

key shape:torch.Size([6, 2])
value shape:torch.Size([6, 2])


In [ ]:
keys_2 = keys[1]
attention_score22 = query2.dot(keys_2)
print(attention_score22) # score for 2nd to itself

tensor(1.8524)


In [ ]:
attention_scores_2 = query2 @keys.T
print(attention_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [ ]:
d_k = keys.shape[1]
attention_weights = torch.softmax((attention_scores_2/d_k**0.5),dim = -1)
print(attention_weights)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [ ]:
context_vec = attention_weights @ values
print(context_vec)

tensor([0.3061, 0.8210])


# Implementing this in compact Self attention Class

In [ ]:
from torch import nn

class SelfAttention_v1(nn.Module):
  def __init__(self,d_in,d_out):
    super().__init__()
    self.W_query = nn.Parameter(torch.rand(d_in,d_out),requires_grad = True)
    self.W_key = nn.Parameter(torch.rand(d_in,d_out),requires_grad = True)
    self.W_value = nn.Parameter(torch.rand(d_in,d_out),requires_grad = True)

  def forward(self,x):
    self.query = x@ self.W_query
    self.key = x @ self.W_key
    self.value = x@self.W_value
    self.attscores = self.query @ self.key.T
    self.d = x.shape[-1]
    self.attweights = torch.softmax((self.attscores/self.d**0.5),dim= -1)
    self.output = self.attweights @ self.value
    return self.output


In [ ]:
torch.manual_seed(123)
selfatt = SelfAttention_v1(d_in = 3,d_out = 2)
selfatt(inputs)

tensor([[0.2961, 0.7970],
        [0.3016, 0.8104],
        [0.3013, 0.8098],
        [0.2921, 0.7873],
        [0.2904, 0.7833],
        [0.2956, 0.7959]], grad_fn=<MmBackward0>)

In [ ]:
class SelfAttention_v2(nn.Module):
  def __init__(self,
               d_in,d_out,qkv_bias = False):
    super().__init__()
    self.W_query = nn.Linear(d_in,d_out,bias = qkv_bias)
    self.W_keys = nn.Linear(d_in,d_out,bias = qkv_bias)
    self.W_values = nn.Linear(d_in,d_out,bias = qkv_bias)

  def forward(self,x):
    queris = self.W_query(x)
    keys = self.W_keys(x)
    values = self.W_values(x)
    dim = x.shape[1]
    attn_scores = queris@keys.T
    attn_weights = torch.softmax((attn_scores/dim**0.5),dim = -1)
    output = attn_weights@values
    return output


In [ ]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in = 3,d_out = 2)
print(sa_v2(inputs))

tensor([[-0.0745,  0.0704],
        [-0.0752,  0.0697],
        [-0.0752,  0.0696],
        [-0.0762,  0.0681],
        [-0.0765,  0.0677],
        [-0.0757,  0.0688]], grad_fn=<MmBackward0>)


# Hiding Future words with Causal attention

In [ ]:
keys = sa_v2.W_keys(inputs)
queries = sa_v2.W_query(inputs)
# print(keys)
d = keys.shape[1]
attn_scores = queries @ keys.T
attn_weights = torch.softmax((attn_scores/d**0.5),dim = -1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [ ]:
context_length = attn_score.shape[0]
mask_simple = torch.tril(torch.ones(context_length,context_length))
mask_simple

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])

In [ ]:
masked_sample = attn_weights * mask_simple
masked_sample

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)

In [ ]:
masked_sample_norm = masked_sample/masked_sample.sum(dim = -1)
masked_sample_norm

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0625, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0595, 0.4484, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.9727, 0.4505, 0.3114, 0.2319, 0.0000, 0.0000],
        [0.9527, 0.4510, 0.3116, 0.2344, 0.1971, 0.0000],
        [1.0070, 0.4495, 0.3109, 0.2276, 0.1979, 0.1529]],
       grad_fn=<DivBackward0>)

We can do it directly if we add the -inf to the tril

In [ ]:
mask = torch.triu(torch.ones(context_length,context_length),diagonal = 1)
masked = attn_scores.masked_fill(mask.bool(),-torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [ ]:
masked_normal_scores = torch.softmax((masked/d**0.5),dim = -1)
masked_normal_scores

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)

# Masking additonal attention weights with dropout

In [ ]:
# Test how dropout works
context_length = inputs.shape[0]
example = torch.ones(context_length,context_length)
drop = torch.nn.Dropout(p =0.5)
droped = drop(example)
print(droped)

tensor([[2., 0., 2., 2., 2., 2.],
        [2., 0., 2., 2., 0., 2.],
        [0., 2., 2., 0., 2., 0.],
        [2., 2., 2., 0., 0., 0.],
        [0., 0., 2., 2., 0., 2.],
        [0., 2., 2., 2., 0., 2.]])


In [ ]:
droped_masked_normal_scores = drop(masked_normal_scores)
print(droped_masked_normal_scores)

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.4925, 0.4638, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3968, 0.3775, 0.3941, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3058]],
       grad_fn=<MulBackward0>)


In [ ]:
batch = torch.stack((inputs,inputs),dim = 0)
print(batch),print(batch.shape)

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])
torch.Size([2, 6, 3])


(None, None)

In [ ]:
class CausalAttention(nn.Module):
  def __init__(self,d_in,d_out,context_length,dropout = 0,
               qkv_bias = False):
    super().__init__()
    self.W_query = nn.Linear(d_in,d_out, bias = qkv_bias)
    self.W_keys = nn.Linear(d_in,d_out, bias = qkv_bias)
    self.W_values = nn.Linear(d_in,d_out, bias = qkv_bias)
    self.dropout = nn.Dropout(dropout)
    self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

  def forward(self,x):
    d,num_tokens,dim = x.shape
    query = self.W_query(x)
    keys = self.W_keys(x)
    values = self.W_values(x)
    # print(query.shape)
    # print(keys.transpose(1,2).shape)
    attention_scores = query@keys.transpose(1,2)
    # print(attention_scores)
    attention_scores.masked_fill_(
        self.mask.bool()[:num_tokens,:num_tokens],-torch.inf
    )
    attn_weights = torch.softmax(
        attn_score/keys.shape[-1]**0.5,dim =-1
    )
    attention_weights = self.dropout(attn_weights)
    context_vec = attention_weights @ values
    return context_vec
    # print(attention_scores)



In [ ]:
torch.manual_seed(123)
context_length = batch.shape[1]
casa = CausalAttention(d_in = 3,d_out = 2,context_length = context_length)
casa(batch)

tensor([[[-0.5417, -0.0965],
         [-0.5539, -0.1171],
         [-0.5537, -0.1170],
         [-0.5422, -0.1144],
         [-0.5418, -0.1117],
         [-0.5450, -0.1160]],

        [[-0.5417, -0.0965],
         [-0.5539, -0.1171],
         [-0.5537, -0.1170],
         [-0.5422, -0.1144],
         [-0.5418, -0.1117],
         [-0.5450, -0.1160]]], grad_fn=<UnsafeViewBackward0>)

# MultiHeaded Attention Wrapper

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self,d_in,d_out,context_length,dropout,qkv_bias,num_heads):
    super().__init__()

    self.heads = nn.ModuleList(
        [CausalAttention(d_in,d_out,context_length,dropout,qkv_bias) for _ in range(num_heads)]
    )

  def forward(self,x):
    return torch.cat([head(x) for head in self.heads],dim = -1)

In [ ]:
torch.manual_seed(123)
context_length = batch.shape[1]
d_in, d_out = 3,2

mha = MultiHeadAttention(d_in,d_out,context_length = context_length,dropout = 0,qkv_bias = False,num_heads = 2)


In [ ]:
mha(batch)

tensor([[[-0.5417, -0.0965,  0.5212,  0.3448],
         [-0.5539, -0.1171,  0.5301,  0.3647],
         [-0.5537, -0.1170,  0.5300,  0.3643],
         [-0.5422, -0.1144,  0.5190,  0.3566],
         [-0.5418, -0.1117,  0.5201,  0.3509],
         [-0.5450, -0.1160,  0.5210,  0.3608]],

        [[-0.5417, -0.0965,  0.5212,  0.3448],
         [-0.5539, -0.1171,  0.5301,  0.3647],
         [-0.5537, -0.1170,  0.5300,  0.3643],
         [-0.5422, -0.1144,  0.5190,  0.3566],
         [-0.5418, -0.1117,  0.5201,  0.3509],
         [-0.5450, -0.1160,  0.5210,  0.3608]]], grad_fn=<CatBackward0>)

In [ ]:
class MultiheadedAttention(nn.Module):
  def __init__(self,d_in,d_out,num_heads,dropout = 0,qkv_bias = False):
    super().__init__()
    self.out_dim = d_out//num_heads
    self.W_query = nn.Linear(d_in,d_out, bias = qkv_bias)
    self.W_keys = nn.Linear(d_in,d_out, bias = qkv_bias)
    self.W_values = nn.Linear(d_in,d_out, bias = qkv_bias)
    self.num_heads = num_heads
    self.out_proj = nn.Linear(d_out,d_out)
    self.d_out = d_out
    self.dropout = nn.Dropout(p = dropout)
    self.register_buffer('mask',
                         torch.triu(torch.ones(context_length,context_length),
                                    diagonal = 1))

  def forward(self,x):
    b,context_length,dim = x.shape
    queries = self.W_query(x)
    keys = self.W_keys(x)
    values = self.W_values(x)

    keys = keys.view(b,context_length,self.num_heads,self.out_dim)
    queries = queries.view(b,context_length,self.num_heads,self.out_dim)
    values = values.view(b,context_length,self.num_heads,self.out_dim)
    # print(values.shape)
    keys = keys.transpose(1,2)
    queries = queries.transpose(1,2)
    values = values.transpose(1,2)

    attention_scores = queries @ keys.transpose(2,3)
    # print(attention_scores)
    mask_bool = self.mask.bool()[:context_length, :context_length]
    # print(mask_bool)
    attention_scores.masked_fill_(mask_bool,-torch.inf)
    # print(attention_scores)
    attention_weights = torch.softmax(attention_scores/keys.shape[-1]**0.5,dim = -1)
    # print(attention_weights)
    # print(values.shape)
    context_vector = (attention_weights @ values).transpose(1,2)
    # print(context_vector)
    context_vector = context_vector.contiguous().view(b,context_length,self.d_out)
    context_vector = self.out_proj(context_vector)
    return context_vector



    # return attention_scores



In [ ]:
torch.manual_seed(123)
c_vect = MultiheadedAttention(d_in,d_out,num_heads = 2)
c_vect(batch)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)